In [1]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
import numpy as np
from gspread import Client
from gspread_dataframe import set_with_dataframe
import urllib
import sqlalchemy
import pyodbc
import sys
import geopandas as gpd
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config

In [2]:
config = get_config('c:\Projects\config.ini')
driver = config['srv']['driver']
server = config['srv']['server']
data_parks = config['srv']['data_parks']
dwh = config['db']['dwh']
cred_file = config['google']['path_to_file']

# Read in Google Sheets

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [4]:
session = create_assertion_session(cred_file, scope)

In [5]:
gc = Client(None, session)

In [6]:
# gc = gspread.authorize(credentials)

In [7]:
wks = gc.open("Crowds_Combined").sheet1

In [8]:
wks2 = gc.open("DailyTasks_WebMerc_Centroids").sheet1

In [9]:
# headers_wks = gc.open("Report_Headers").sheet1

In [10]:
data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
df.head()

,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,6/9/2020 12:27:25,R-01,12,No,Did not approach the crowd; the crowd remains,Open field/multi-purpose play area,Multiple golfers on Silver Lake Golf Course.,R022-ZN02 | Silver Lake Park Zone 2,Staten Island
1,6/8/2020 19:32:53,Q-11,25,No,Did not approach the crowd; the crowd remains,Open field/multi-purpose play area,Soccer game,Q103 | Kennedy Playground,Queens
2,6/8/2020 16:58:09,Q-14,5,No,Approached the crowd; they complied with instr...,Skate Park,B108 hockey,Q163-01 | Rockaway Beach Boardwalk-Hockey Rink,Queens
3,6/8/2020 16:10:52,Q-05A,30,No,Did not approach the crowd; the crowd remains,Baseball field,Ballfield game starting on field behind playgr...,Q020-02 | Highland Park-Upper Highland Playground,Queens
4,6/8/2020 15:38:33,B-10,34,Yes,Did not approach the crowd; the crowd remains,Playground,"Swings, handball court, playground, etc.",B028-03 | Dyker Beach Park-Dyker Playground,Brooklyn


In [11]:
geom = wks2.get_all_values()
headers_geom = geom.pop(0)

df2 = pd.DataFrame(geom, columns=headers_geom)
df2.head()

,PROPERTY_I,DESCRIPTIO,DISTRICT,DESC_LOCAT,ESRI_OID,Latitiude,Longitude,Borough
0,Q102,Juniper Valley Park,Q-05,Q102 | Juniper Valley Park,1,40.7202,-73.8804,Queens
1,Q024,Kissena Park,Q-07,Q024 | Kissena Park,2,40.7454,-73.8048,Queens
2,Q347,Colden Playground,Q-07,Q347 | Colden Playground,3,40.7702,-73.8272,Queens
3,Q121,Detective Keith L Williams Park,Q-12,Q121 | Detective Keith L Williams Park,4,40.7029,-73.7844,Queens
4,Q375,Willets Point Playground,Q-07,Q375 | Willets Point Playground,5,40.7838,-73.7964,Queens


In [12]:
df2.rename(columns = {'Latitiude':'Latitude'},inplace=True)

In [13]:
df2['Latitude'] = df2['Latitude'].astype(float)

In [14]:
df2['Longitude'] = df2['Longitude'].astype(float)

In [15]:
geom_gdf = gpd.GeoDataFrame(df2,
                 geometry=gpd.points_from_xy(df2.Longitude, df2.Latitude),
                 crs={'init': 'epsg:4326'})

In [16]:
geom_gdf.head()

,PROPERTY_I,DESCRIPTIO,DISTRICT,DESC_LOCAT,ESRI_OID,Latitude,Longitude,Borough,geometry
0,Q102,Juniper Valley Park,Q-05,Q102 | Juniper Valley Park,1,40.7202,-73.8804,Queens,POINT (-73.88039999999999 40.7202)
1,Q024,Kissena Park,Q-07,Q024 | Kissena Park,2,40.7454,-73.8048,Queens,POINT (-73.8048 40.7454)
2,Q347,Colden Playground,Q-07,Q347 | Colden Playground,3,40.7702,-73.8272,Queens,POINT (-73.8272 40.7702)
3,Q121,Detective Keith L Williams Park,Q-12,Q121 | Detective Keith L Williams Park,4,40.7029,-73.7844,Queens,POINT (-73.78440000000001 40.7029)
4,Q375,Willets Point Playground,Q-07,Q375 | Willets Point Playground,5,40.7838,-73.7964,Queens,POINT (-73.79640000000001 40.7838)


In [17]:
MO_Crowds = geom_gdf[['DESC_LOCAT','geometry']].merge(df, how = 'right', left_on = 'DESC_LOCAT', right_on = 'Site')

In [18]:
MO_Crowds.head()

,DESC_LOCAT,geometry,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),5/25/2020 9:37:24,Q-05,100,Yes,Did not approach the crowd; the crowd remains,Track/field area,,Q102 | Juniper Valley Park,Queens
1,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/27/2020 15:07:25,Q-05,200,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
2,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/26/2020 14:37:04,Q-05,12,No,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
3,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),3/26/2020 14:36:46,Q-05,8,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
4,Q024 | Kissena Park,POINT (-73.8048 40.7454),5/4/2020 9:16:57,Q-07,80,No,Did not approach the crowd; the crowd remains,Walking path,,Q024 | Kissena Park,Queens


# change datatype of Timestamp to datetime

In [19]:
MO_Crowds['Timestamp'] = pd.to_datetime(MO_Crowds['Timestamp'], infer_datetime_format=True)

In [20]:
MO_Crowds

,DESC_LOCAT,geometry,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
0,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-05-25 09:37:24,Q-05,100,Yes,Did not approach the crowd; the crowd remains,Track/field area,,Q102 | Juniper Valley Park,Queens
1,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-27 15:07:25,Q-05,200,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
2,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-26 14:37:04,Q-05,12,No,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
3,Q102 | Juniper Valley Park,POINT (-73.88039999999999 40.7202),2020-03-26 14:36:46,Q-05,8,Yes,Did not approach the crowd; the crowd remains,,,Q102 | Juniper Valley Park,Queens
4,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-05-04 09:16:57,Q-07,80,No,Did not approach the crowd; the crowd remains,Walking path,,Q024 | Kissena Park,Queens
5,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-04-19 12:21:33,Q-07,5,No,Did not approach the crowd; the crowd remains,Soccer field,Kissena soccer field team play \n,Q024 | Kissena Park,Queens
6,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-26 12:16:08,Q-07,6,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
7,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-25 17:05:47,Q-07,6,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
8,Q024 | Kissena Park,POINT (-73.8048 40.7454),2020-03-25 16:58:29,Q-07,5,No,Did not approach the crowd; the crowd remains,,,Q024 | Kissena Park,Queens
9,Q012B | John Golden Park,POINT (-73.76819999999999 40.7739),2020-05-27 16:16:03,Q-07A,25,No,Did not approach the crowd; the crowd remains,Baseball field,Ongoing unauthorized group play at Field 4,Q012B | John Golden Park,Queens


In [21]:
con_string = 'Driver={SQL Server};Server=dpr-vpipm001;Database=dwh;Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [22]:
def to_wkt(row):
    return row.wkt

In [23]:
MO_Crowds.geometry

0        POINT (-73.88039999999999 40.7202)
1        POINT (-73.88039999999999 40.7202)
2        POINT (-73.88039999999999 40.7202)
3        POINT (-73.88039999999999 40.7202)
4                  POINT (-73.8048 40.7454)
5                  POINT (-73.8048 40.7454)
6                  POINT (-73.8048 40.7454)
7                  POINT (-73.8048 40.7454)
8                  POINT (-73.8048 40.7454)
9        POINT (-73.76819999999999 40.7739)
10       POINT (-73.76819999999999 40.7739)
11       POINT (-73.76819999999999 40.7739)
12       POINT (-73.76819999999999 40.7739)
13       POINT (-73.87479999999999 40.7588)
14       POINT (-73.91079999999999 40.7475)
15                 POINT (-73.8074 40.7705)
16                 POINT (-73.8074 40.7705)
17       POINT (-73.90260000000001 40.7396)
18       POINT (-73.90260000000001 40.7396)
19       POINT (-73.90260000000001 40.7396)
20       POINT (-73.90260000000001 40.7396)
21       POINT (-73.90260000000001 40.7396)
22       POINT (-73.902600000000

In [24]:
len(df)

1584

In [25]:
len(MO_Crowds)

1585

In [26]:
MO_Crowds_to_publish = MO_Crowds[~pd.isnull(MO_Crowds['geometry'])].copy()

In [27]:
MO_Crowds_to_publish['geometry'] = MO_Crowds_to_publish['geometry'].apply(to_wkt)

In [28]:
MO_Crowds_to_publish.sort_values(by='Timestamp')

,DESC_LOCAT,geometry,Timestamp,District,NumberOfPeople,CrowdInPlayground,ActionTakenByParksEmployee,LocationInPark,AdditionalComments,Site,Borough
894,M108T01 | East River Esplanade,POINT (-73.9363 40.7897),2020-03-25 08:07:40,M-15,17,Yes,Approached the crowd; they complied with instr...,,,M108T01 | East River Esplanade,Manhattan
1572,R030-099 | Willowbrook Park-Willowbrook Park,POINT (-74.15696 40.6007),2020-03-25 10:44:20,R-02,18,No,Approached the crowd; they ignored the employee,,,R030-099 | Willowbrook Park-Willowbrook Park,Staten Island
1318,R045-01 | Schmul Park-Schmul Park,POINT (-74.187 40.5894),2020-03-25 10:49:03,R-02,8,Yes,Approached the crowd; they ignored the employee,,,R045-01 | Schmul Park-Schmul Park,Staten Island
583,R005 | Clove Lakes Park,POINT (-74.11369999999999 40.6195),2020-03-25 10:49:53,R-01,6,No,Did not approach the crowd; the crowd remains,,,R005 | Clove Lakes Park,Staten Island
1127,R005-ZN01 | Clove Lakes Park Zone 1,POINT (-74.1097 40.6164),2020-03-25 10:54:28,R-01,10,No,Did not approach the crowd; the crowd remains,,,R005-ZN01 | Clove Lakes Park Zone 1,Staten Island
1310,R005-01 | Clove Lakes Park-Clove Lakes Park Pl...,POINT (-74.10599999999999 40.6185),2020-03-25 10:58:02,R-01,10,Yes,Did not approach the crowd; the crowd remains,,,R005-01 | Clove Lakes Park-Clove Lakes Park Pl...,Staten Island
1308,R005-15 | Cloves Lakes Park-Saturn Play Equipment,POINT (-74.1211 40.6271),2020-03-25 11:02:16,R-01,2,Yes,Did not approach the crowd; the crowd remains,,,R005-15 | Cloves Lakes Park-Saturn Play Equipment,Staten Island
524,B022 | Middleton Playground,POINT (-73.9554 40.7007),2020-03-25 11:06:58,B-01,75,Yes,Did not approach the crowd; the crowd remains,,,B022 | Middleton Playground,Brooklyn
1126,R005-ZN01 | Clove Lakes Park Zone 1,POINT (-74.1097 40.6164),2020-03-25 11:24:16,R-01,2,No,Did not approach the crowd; the crowd remains,,,R005-ZN01 | Clove Lakes Park Zone 1,Staten Island
1125,R005-ZN01 | Clove Lakes Park Zone 1,POINT (-74.1097 40.6164),2020-03-25 11:31:08,R-01,7,No,Approached the crowd; they complied with instr...,,,R005-ZN01 | Clove Lakes Park Zone 1,Staten Island


In [29]:
MO_Crowds_to_publish.to_sql('tbl_MO_crowds', engine, if_exists='replace')

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\mssql\base.py:2554: SAWarning: Did not recognize type 'geometry' of column 'geometry'
  % (type_, name)


In [30]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                     ';Database=' + dwh + ';Trusted_Connection=Yes;')

In [31]:
cursor = con.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_MO_crowds] 
    alter column geometry geometry
    """)
con.commit()
cursor.close()